In [ ]:
# init
import importlib
import tTsTGrpUtils as tsutil
import utils_plots as uplots
import os
import sys
sys.path.append("/host/verges/tank/data/daniel/3T7T/z/code/analyses/hippunfold_toolbox/")
from hippunfold_toolbox import plotting
import numpy as np
import pandas as pd

test = False # for quick testing

# Data paths
root = "/host/verges/tank/data/daniel/3T7T/z/outputs/"
pth_05c_winD = "05c_stats_winD_20Oct2025-110828.pkl" # Has all data from above maps as well. D score btw px and ctrl z-scores
pth_05d_btwD = "05d_btwD_20Oct2025-114530.pkl" # difference in D scores between studies


#dl_winD = tsutil.loadPickle(root + pth_05c_winD)
dl_btwD = tsutil.loadPickle(root + pth_05d_btwD)
dl_winD = tsutil.loadPickle(root + pth_05c_winD)
demo = "/host/verges/tank/data/daniel/3T7T/z/outputs/03c_demoPths_clean_18Oct2025-153812.csv"
#qc_table = "/host/verges/tank/data/daniel/3T7T/z/outputs/old_QCtables/03a_qc_table_09Oct2025-094401.csv"
df_demo = pd.read_csv(demo)
#qc_table = pd.read_csv(qc_table)

In [ ]:
# merge QC table with df_demo, matching on UID, study, ID, SES, Date
#df_demo = df_demo.merge(qc_table, on=['UID', 'study', 'SES', 'Date'], how='left')

df_demo['UID_ID_SES_3T'] = df_demo.apply(lambda row: f"{row['UID']}_{row['MICS_ID']}_{row['SES']}", axis=1)
df_demo['UID_ID_SES_7T'] = df_demo.apply(lambda row: f"{row['UID']}_{row['PNI_ID']}_{row['SES']}", axis=1)
df_demo_3T = df_demo[df_demo['study'] == '3T']
df_demo_7T = df_demo[df_demo['study'] == '7T']
df_demo_3T.columns.tolist()

In [ ]:
#tsutil.print_dict(dl_winD)
dl_winD_filt = tsutil.filt_dl(dl_winD, 'feature', 'T1map')

lens_ctrl = []
lens_tle = []
for item in dl_winD_filt:
    ctrl_ids = item['ctrl_IDs']
    tle_r_ids = item['TLE_R_IDs']
    tle_l_ids = item['TLE_L_IDs']

    len_ctrl = len(ctrl_ids)
    len_tle = len(tle_r_ids) + len(tle_l_ids)
    lens_ctrl.append(len_ctrl)
    lens_tle.append(len_tle)

# find index of max length
max_len_ctrl = max(lens_ctrl)
max_len_tle = max(lens_tle)
idx_max_ctrl = lens_ctrl.index(max_len_ctrl)
idx_max_tle = lens_tle.index(max_len_tle)
print(lens_ctrl)
print(lens_tle)
print(f"Max ctrl len: {max_len_ctrl} at index {idx_max_ctrl}")
print(f"Max TLE len: {max_len_tle} at index {idx_max_tle}")

tsutil.printItemMetadata(dl_winD_filt[0], idx = 0)
tsutil.printItemMetadata(dl_winD_filt[1], idx = 1)
tsutil.printItemMetadata(dl_winD_filt[44], idx = 44)
tsutil.printItemMetadata(dl_winD_filt[45], idx =45)

In [ ]:
ctrl_IDs_3T = dl_winD[0]['ctrl_IDs']
ctrl_IDs_7T = dl_winD[1]['ctrl_IDs']

TLE_IDs_3T = dl_winD[44]['TLE_L_IDs'] + dl_winD[44]['TLE_R_IDs']
TLE_IDs_7T = dl_winD[45]['TLE_L_IDs'] + dl_winD[45]['TLE_R_IDs']
print()

In [ ]:
ctrl_IDs_7T + TLE_IDs_7T

In [ ]:
df_demo_3T = df_demo[df_demo['study'] == '3T']
df_demo_7T = df_demo[df_demo['study'] == '7T']

df_demo_final_3T = df_demo_3T[df_demo_3T['UID_ID_SES_3T'].isin(ctrl_IDs_3T + TLE_IDs_3T)]
df_demo_final_7T = df_demo_7T[df_demo_7T['UID_ID_SES_7T'].isin(ctrl_IDs_7T + TLE_IDs_7T)]

df_demo_final = df_demo_final_3T[['UID', 'MICS_ID', 'PNI_ID', 'age', 'sex', 'education','ethnicity', 'grp', 'grp_detailed', 'Histopatholgy', 
                                 'Drug resistant epilepsy at time of EMU admission', 'Seizure onset (yr)', 'Dx at EMU discharge ']]

dateDif_d = pd.Series(dtype=float)
dateDif_wks = pd.Series(dtype=float)
dateDif_yrs = pd.Series(dtype=float)
for uid in df_demo_final_3T['UID']:
    Date_3T = df_demo_final_3T[df_demo_final_3T['UID'] == uid]['Date'].values[0]
    Date_7T = df_demo_final_7T[df_demo_final_7T['UID'] == uid]['Date'].values[0]
    date_diff_days = np.abs((pd.to_datetime(Date_7T,dayfirst = True) - pd.to_datetime(Date_3T, dayfirst = True)).days)
    date_diff_wks = date_diff_days / 7
    date_diff_yrs = date_diff_days / 365.25
    #print(f"UID: {uid}, Date 3T: {Date_3T}, Date 7T: {Date_7T}, Date Difference (days): {date_diff_days}")
    dateDif_d[uid] = date_diff_days
    dateDif_wks[uid] = date_diff_wks
    dateDif_yrs[uid] = date_diff_yrs
#print(dateDif)
df_demo_final['dateDif'] = df_demo_final['UID'].map(dateDif)
df_demo_final['dateDif_wks'] = df_demo_final['UID'].map(dateDif_wks)
df_demo_final['dateDif_yrs'] = df_demo_final['UID'].map(dateDif_yrs)

In [ ]:
# remove rows with grp_detailed == "TLE_BL"
df_demo_final = df_demo_final[df_demo_final['grp_detailed'] != 'TLE_BL']
df_demo_final[df_demo_final['grp'] == 'CTRL']

In [ ]:
# note age at 3T
# based on this df, create a summary statistics table
# for categorical variables, show counts and percentages
# Create summary statistics for both numerical and categorical variables

# Create summary statistics grouped by 'grp' column
print("=== SUMMARY STATISTICS BY GROUP ===")

summary_stats = []

for grp in df_demo_final['grp'].unique():
    print(f"\n{'='*50}")
    print(f"GROUP: {grp}")
    print(f"{'='*50}")
    
    grp_data = df_demo_final[df_demo_final['grp'] == grp]
    print(f"Total participants: {len(grp_data)}")
    
    print(f"\n--- NUMERICAL VARIABLES ---")
    numerical_cols = grp_data.select_dtypes(include=[np.number]).columns
    num_stats = grp_data[numerical_cols].describe()
    print(num_stats)
    
    print(f"\n--- CATEGORICAL VARIABLES ---")
    categorical_cols = grp_data.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        if col != 'grp':  # Skip the grouping variable itself
            print(f"\n{col}:")
            value_counts = grp_data[col].value_counts(dropna=False)
            percentages = grp_data[col].value_counts(normalize=True, dropna=False) * 100
            summary_df = pd.DataFrame({
                'Count': value_counts,
                'Percentage': percentages.round(1)
            })
            print(summary_df)
            
            # Store for CSV export
            for idx, row in summary_df.iterrows():
                summary_stats.append({
                    'Group': grp,
                    'Variable': col,
                    'Value': idx,
                    'Count': row['Count'],
                    'Percentage': row['Percentage']
                })

# Add numerical statistics to summary_stats
for grp in df_demo_final['grp'].unique():
    grp_data = df_demo_final[df_demo_final['grp'] == grp]
    numerical_cols = grp_data.select_dtypes(include=[np.number]).columns
    num_stats = grp_data[numerical_cols].describe()
    
    for col in numerical_cols:
        for stat in num_stats.index:
            summary_stats.append({
                'Group': grp,
                'Variable': col,
                'Value': f'{stat}',
                'Count': num_stats.loc[stat, col],
                'Percentage': ''
            })

# Convert to DataFrame and save as CSV
summary_stats_df = pd.DataFrame(summary_stats)
summary_stats_df.to_csv('/host/verges/tank/data/daniel/3T7T/z/outputs/summary_statistics_by_group.csv', index=False)
print(f"\nSummary statistics saved to: /host/verges/tank/data/daniel/3T7T/z/outputs/summary_statistics_by_group.csv")


In [ ]:
# create a df with jut TLE
demo_TLE_3T = df_demo_3T[df_demo_3T['UID_ID_SES_3T'].isin(TLE_IDs_3T)]
demo_TLE_7T = df_demo_7T[df_demo_7T['UID_ID_SES_7T'].isin(TLE_IDs_7T)]

# Combine the TLE dataframes from both studies
demo_TLE = pd.concat([demo_TLE_3T, demo_TLE_7T], ignore_index=True)
demo_TLE = demo_TLE[demo_TLE['grp_detailed'] != 'TLE_BL']
demo_TLE['ID'] = demo_TLE.apply(lambda row: row['MICS_ID'] if row['study'] == '3T' else row['PNI_ID'], axis=1)
demo_TLE.columns
# save to df
demo_TLE.to_csv('/host/verges/tank/data/daniel/3T7T/z/outputs/001_demo_TLE_volPths.csv', index=False)

In [ ]:
demo_TLE.columns.tolist()